# Advanced LLM Prompt Cache Architecture Overview
We'll cover several high-level sections from Hardware to Software and Algorithms with intrigue in-between.

## Hardware-Level Caching

### Leveraging the Chip Layer Caches
- Big Cache, sure... L1, L2, L3
- Choosing your hardware purposefully, eg "EPYC MegaL3 Cache"* vs "Xeon MAX"**

- [*] Not its formal marketing name, but close
- [**] This is its actual marketing name

### DRAM Faster All The Time
DDR4 is no more.. we go with DDR5 and soon DDR6, and we'll see HMB, HMB2, and other designs in different ways.

#### Concerns for DRAM
- NUMA, and it applies to chiplet designs just as much as it applies to multi-socket motherboards.
- Interleaved memory: and how it can be used to improve performance, improve high-availability, as well as concerns about degrading performance if the design does not fit the requirements.
- Memory channel density: this can increase total DRAM density, though in some architectures there is a DRAM speed penalty for populating more than 1 DIMM per Channel (1DPC).


### NVMe are not all equal
- U.2, U.3, AIC, or Rulers? Different specs all around, and their interconnects to the PCIe bus make a substantial difference.
- SLC on there, is it an SLC cache or is the full NAND array made of SLC

### RDMA and Network Shared Memory
Is OpenMPI ever enough? Is RDMA what we want? Why not Myrcom

#### Interconnect Designs for Memory Nets
As a starting point, the Torus has been a highly performing base architecture for Top-500 supercomputers. More on that here:
- https://en.wikipedia.org/wiki/Torus_interconnect#Performance

Variations on the theme, including SeaStar and Gemini interconnects are described here:
- https://github.com/jeffhammond/HPCInfo/blob/master/docs/Cray.md

#### Memory Passing Interface - aka MPI
What would be do with a shared memory network if not pass data back and forth between compute nodes? Yes, and in a standardized manner, where MPI offers plenty of examples. This has been an evolving field of computer science, which deserves a variety of source code examples and tutorials:
- From the great Jeff Hammond: https://github.com/jeffhammond/HPCInfo/tree/master/mpi
- Lawrence Livermore Labs gets a mention: https://hpc-tutorials.llnl.gov/mpi/


### Multi-Node Distributed Caching
This necessarily involves an amalgam of the various cache layers and technologies, and so we'll simply consider this to be the default when dealing with clusters of KVUs.

#### Shared Considerations on Timing
This applies to shared and non-shared resource clusters, and it is the resource of Time.
- Are we using a strict Precision Time Protocol system, Stratum-1 or are we leveraging NTP's global pool for Stratum-2 sync?
- Are the NICs the Source of Truth for Network Time, or do we have a separate SMA connection on the wire for PTP data distribution?

#### Considerations on Shared-Nothing Clusters
- Which layer controls cache coherency across the cluster if not the shared-resource controller service?
- Is there cache-coherency or is this a broadcast-type information domain where the First Responder wins?
- If we share nothing across the nodes, how do we prevent split-brain issues where more than one individual node tries to answer first and wastes resources when multiple nodes are trying for the same connection?


#### Considerations on Shared-Resource Clusters
- Scaling algos, pre-determined calculations abound.
- Must control for N-th order latency and bandwidth for resource communications over the fabric.
- Do we have a single shared L2 domain fabric with ALCs, or a segmented L3, or layered L3 with VxLAN or Mesh-VPN/type networks-within-networks?
- How are the switching layers impacted on node scaling baselines?
- Are we using a 1:1 mapping for ports between node-to-node or node-to-spine or node-to-leaf?
- And so forth...

#### Hybrid Cluster Designs
This is a combination, in simpler terms it's effectively a "Shared Some-Things" instead of 100% None vs 100% Everything (Disk, Memory, Network, CPU cycles via Swarm / OpenMP).

#### Exceptions to Multi-Node
The exceptions include hypothetical designs like the following, and others of one's imagination. Real-world examples abound, but are out of scope for this document at present.

- Micro-Cluster: A single CS-N with 2-4U compute nodes, vertically scaled for high-density resources per node, leveraging 10s-100s of VMs per single-node.
    - Each single physical node could be deployed as a single dedicated tenant, similar in concept to using single-physical L2 network domains for dis-aggregation and strong physical isolation of systems when logical partitioning is not relevant by CSO requirements or DoD spec or similar architectural design requirement.
    - In this design the VMs could participate in aggregation-level distributed caching internally to the physical node, but would not leverage hardware offloads directly on NICs or DPUs unless distributed via SR-IOV or NPAR style controls within the single-node's OS and kernel + CGroups security layers.


## Application Level Caching
The following list can be expanded into sub-sections if desired.

- Semantic Cache Algorithms
- Library Layer Cache Optimizers
- Template and Prefix Caching
- Multi-Turn Conversions
- RAG and Top-K Concerns
- Vector Embeds, aka The Matryoshka Method
- SDK Optimizers Per-Platform


## The Many Methods of Popularized Cache Algorithms

### Most Recently Used (MRU)
MRU evicts the most recently used item, which can be useful in page-scanning engines or cyclical-control loop caches. This operates under the assumption that the MRU item was scanned and accessed, but then will not be referenced again until the next full-loop or full-cycle/page. It is useful in very specific circumstances, or as a layered algo with trigger calls to enable/disable its functionality as-needed.

. MRU is only beneficial in specific patterns (like scanning cyclical patterns). It’s generally not used as a primary policy in caching systems for LLMs, except possibly in some layered buffer situations.

### Least Recently Used (LRU)
Possibly the most commonly seen cache algo in the wild, LRU evicts items accessed the longest time ago, operating via the assumption that if it hasn’t been used in a while then it’s less likely to be needed soon. Downside: LRU only tracks recency, not frequency of use, and has no weighting controls.

### LRU variants (2Q, LRU/K, etc)
- 2Q is a strategy that introduces a probationary queue and a protected queue
- LRU/K and LRU/2 track the k-th most recent access rather than the last access when making eviction decisions

### Least Frequently Used (LFU)
Evicts the item with the lowest access count to better capture long-term hot items vs cold items, another entry in the list of comparatively simple cache algos.

### Random
As it stands, it's A method, but not an often-best method for cache controls.

### First In First Out (FIFO)
A circular buffer type cache policy, similar to Round-Robin method in Load-Balancing controls. Very simple, and sometimes KISS is all that's needed.

### Clock Second Chance (CSCh)
An approximation of LRU used in OS kernel page replacement, which arranges pages in a circular buffer like a clock and gives a "second chance" to pages that have been accessed recently by marking them before eviction. Effectively this is a Round-Robin method with a simple weighting system.

### Adaptive Replacement Cache (ARC)
ARC is a highly regarded algorithm that adapts between LRU and LFU dynamically, algorithmically defined via tunables, used in the ZFS filesystem's DRAM and L2 data segments, as well as various applications which require real-time analysis and automated tuning of the caching system. This is the most advanced of the cache controls described here.

### Segmented LRU (S/LRU)
Takes the LRU method and splits cache entries into segments, referred to as "protected vs probationary".

### Window TinyLFU (Wt/LFU)
An advancement upon the standard LFU policy which uses a small LRU "windowing controller" and adapts the concept of "TinyLFU" for its cache admission policy design. These actions can be modeled by ARC and improved upon dynamically, where-as the non-ARC methods of Wt/LFU may become stale over time or induce inefficiency which is difficult to change during production use - unless real-time controls are implemented.

## Innovative Cache Designs

- Learning-Based Cache Evictions
- Content-Based Caching
- Hierarchical Cooperative Caching
- Disaggregated Network Caching with Memory Pooling
- Multi-Modal Caching, Internalized
- Write-Cache Persistence Cycles
- Dynamic Cache Partitioning
- Cache-within-a-Cache, using Caches to Cache other Cache Info

## Cache-Object Invalidation Strategies
- Common strategies include time-to-live (TTL)
- Object Not Found Error (ONFE)

### Cache Propagation Strategies
- First-Order Connection Fan-Out
- Multicast Subnet ACLs
- Unicast Subnet ACLs

### Invalidation Layered Invalidation Methods
- Object Origin Dead on Arrival (OO-DoA)
- Object Origin Dead on Health-Check (OO-DoHC)



## Concerns in Caching Architectures
This topic is extensive, and for the current scope I'll cover the high-level sections. If or when those need further clarifications, then the topics may be expanded upon.

### Stale Data and Invalidation Issues
How do we handle this... it's like asking "Which bear is the best bear?".. not a simple answer.

### Cache Consistency Across Multi-Node Clusters
This involves cache invalidation, stale data, awareness and coherency, split-brain concepts, and everything in-between but not limited to whether we should use fan-out, broadcast, target-steering, traffic-directors, or some manner of snake oil to solve the many concerns.

### The Stampeding Thunderous Herd was Heard on All Nodes
Jokes aside, this ages old concern about connection floods is felt not only on cache node systems involved with mass-invalidation or mass-resync or mass-N actions.

It's a concern on datacenter power policy when an entire rack power cycles without a turn-up plan -- should all nodes immediately power on after power-loss or should there be a randomized counter to ensure that the UPS or PDUs or upstream delivery systems are not slammed all at once and blow caps or smash breakers.

We see this issue in many areas of infrastructure, civil engineering, aerospace engineering, computer science, and even our own neuroendocrine system which we really don't want to flood the brain with catecholamines during a stressful event - because then you get shock, cardiac arrest, vascular issues leading to aneurysm.

### Cache Pollution or Cache-Affinity Overuse
If an algo is tuned incorrectly or inefficiently, we can see certain "hot nodes" being worn our on TBW disk load, or NICs constantly at peak instead of load being distributed across adjacent nodes, or we see de-serialization issues, chunking inefficiency, and so forth.

### Over-Serialization and Transport Cost Overload
Moving data back and forth between nodes, across spine and leafs, through DPU caches, through off-loader chips, etc.. these actions all have costs: power costs, latency inductions, bandwidth usage, compute cycles, etc. If transport is being inefficiently used due to poorly tuned algos, then we see cost increases occuring unnecessarily, and wear rates being exceeded earlier than they would if the cache efficiency were more effective. This is an overall systems engineering optimization and performance engineering analysis side of cache controls.

### Memory Leaks and Resource Locks
As their tites say, these are issues with programming and resource alignments. These can occur in myriad situations, which can cover entire chapters. Such as they are, they can cause extensive issues and preventative measures must be employed to identify their occurrence before production implementation, as well as checks for production to find and resolve or mitigate before large-scale cache propagation occurs.

### Over-Caching of Data and Data-Couplers
Sometimes there's a greater cost and performance efficiency by not caching data across a cluster, and sometimes coupling indirect data or metadata with the original data is more of less effective than its opposite. These topics can be expanded upon if necessary.


## Standardizing the Architecture Design Process, References

We can reference the following concepts for correctness and usage. These are a subset of my preferred methods, primarily high-level approaches which are helpful for narrowing down the decision-arrival process while attempting to ensure adherence to a level of consistency in Correctness-Defined.

- "Markdown Architectural Decision Records", https://adr.github.io/madr/
- "Architecturally significant requirements", https://en.wikipedia.org/wiki/Architecturally_significant_requirements
- "Attribute Driven Design", https://en.wikipedia.org/wiki/Attribute-driven_design
- "Definition of Done", https://ozimmer.ch/practices/2020/05/22/ADDefinitionOfDone.html
- "Definition of Ready", https://ozimmer.ch/practices/2023/12/01/ADDefinitionOfReady.html
- "ADR Templates", https://adr.github.io/adr-templates/
- "Decision Capturing Tools", https://adr.github.io/adr-tooling/